## Features are
## 1) RR average length, mean and median QRS width, RR length standard deviation
## 2) Number of changes in sign of the the slope in the R-R interval (per time)
## 3) standard deviation of slope in the R-R interval as well as average and median of the absolute value of the slopes
## 4) PCA on DWT
## 5) Difference between median and mean heartbeats (the norm of the vector)
## 6) std dev. of the pre-R part of the beat
## 7) HOS descriptor
## 8) k-means of RR lengths
## 9) Autoregression coefficients of order 9s and 10: x(n)  = coeff1 * x(n-1) + coeff2 * x(n-2) +..+ ## coeff9*x(n-9) and x(n)  = coeff1 * x(n-1) + coeff2 * x(n-2) +..+ ## coeff10*x(n-10)

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.exceptions import DataConversionWarning
import warnings
from sklearn.cluster import KMeans
import pandas as pd
from pywt import wavedec
from pywt import waverec
from statsmodels.tsa.stattools import levinson_durbin
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
import pywt
import scipy
#from noiseremove import func0
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import SMOTE
import pylab as pl
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from biosppy.signals import ecg

In [3]:
dfX = pd.read_csv("X_train.csv").iloc[:, 1:]
dfTest = pd.read_csv("X_test.csv").iloc[:, 1:]
dfY = pd.read_csv("y_train.csv").iloc[:, 1:]
XPred = dfTest.iloc[:, 1:].values
y = dfY.values
MAX_SENT = 10e5
X = dfX.fillna(MAX_SENT).values  # fill the nan with sentinel values to prevent error later
X[-1][-1] = MAX_SENT
Xt= dfTest.fillna(MAX_SENT).values  # fill the nan with sentinel values to prevent error later
Xt[-1][-1] = MAX_SENT

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17979) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
y_good = np.where(y !=3)[0]
X_ = X[y_good]
y_ = y[y_good]  
## undoes the previous statements (without having to change the code below)
X_= X
y_ = y

In [5]:
#preds3 = func0().astype(int)
#available = np.where(preds3 != 1)[0]
#Xt = Xt[available]

NameError: name 'func0' is not defined

In [6]:
# maxxl = -9 ==> 17979
def filtersig(rsig):
    padded = np.array([MAX_SENT]*17979)
    sig_ = rsig[rsig < MAX_SENT]
    info = ecg.ecg(signal=sig_, sampling_rate= 300, show=False)
    filt_sig = info[1]
    padded[0:len(filt_sig)] = filt_sig
    return padded

X_filt = np.apply_along_axis(filtersig, axis = 1, arr = X_)
#slope_stats = np.apply_along_axis(slopestat, axis = 1, arr = mix_data)

In [7]:
max_len = -5
def getRpeaks(sig):
    #global max_len 160
    padded = np.array([MAX_SENT]*212)
    sig_ = sig[sig < MAX_SENT]
    info = ecg.ecg(signal=sig_, sampling_rate= 300, show=False)
    rpks= info[2]
    padded[0:len(rpks)] = rpks
    return padded

r_peaks = np.apply_along_axis(getRpeaks, axis = 1, arr = X_)
mix_data2 = np.concatenate((r_peaks, X_filt), axis = 1)

## New: Generate new samples from existing ones
The idea consists of taking two signals of the same class. We then split the two signals in half, and combine them again using the first half of signal A and the second half of signal B to get a new signal. However, to avoid messing up the QRS complexes morphology we use the mean RR interval of the common terminals of the original signals.


In [13]:
def combineSigs(mix_1, mix_2):
    
    rpks1 = mix_1[0:212]
    rpks2 = mix_2[0:212]
    
    rpks1  = np.array(rpks1[rpks1 < MAX_SENT], dtype = int)
    rpks2  = np.array(rpks2[rpks2 < MAX_SENT], dtype = int)
    
    rlen1 = int(len(rpks1)/2)
    rlen2 = int(np.floor(len(rpks2)/2)+1)
    res = np.array([MAX_SENT]*17979)
    
    if (rlen1 < 2 or rlen2 < 2):
        return res
    
    sig1 = mix_1[212:]
    sig2 = mix_2[212:]
    sig1  = sig1[sig1 < MAX_SENT]
    sig2  = sig2[sig2 < MAX_SENT]
    
    mid1_pre = rpks1[rlen1]
    mid1_post = rpks1[rlen1 + 1]
    
    mid2_post = rpks2[rlen2]
    mid2_pre = rpks2[rlen2 - 1]
    
    mid1 = (mid1_pre + mid1_post)/2
    mid2 = (mid2_pre + mid2_post)/2
    
    sig_fin = np.append(sig1[0:int(mid1)], sig2[int(mid2):])
    res[0:len(sig_fin)] = sig_fin
    return res
    
    
def sigincrement(sigclass, sig_idx_start, sig_idx_end):
    global X_, y_
    indices = np.where(y_ == sigclass)[0]
    candidates = len(indices)
    mix_data_cl = mix_data2[indices]   # mix_data2 is r_peaks, X_filt
    for i in range(sig_idx_start, sig_idx_end):
        first_sig = mix_data_cl[i]
        for j in range(i+1, candidates):
            second_sig = mix_data_cl[j]
            new_sig = combineSigs(first_sig, second_sig)
            X_ = np.vstack((X_, new_sig)) if (new_sig[0] < MAX_SENT) else X_
            y_ = np.append(y_, sigclass) if (new_sig[0] < MAX_SENT) else y_

In [ ]:
print(X_.shape)
sigincrement(1, 0, 2)
print(X_.shape)

# filtering and r-peak have to be redone to take into account the new samples
X_filt = np.apply_along_axis(filtersig, axis = 1, arr = X_)
r_peaks = np.apply_along_axis(getRpeaks, axis = 1, arr = X_)
mix_data2 = np.concatenate((r_peaks, X_filt), axis = 1)

(4947, 17979)


## newly added part ends here

In [7]:
min_dist = 1000
max_dist = -5
def getRRstats(rpks):
    #global min_dist, max_dist 1.0, 4050
    rpks = rpks[rpks < MAX_SENT]
    diff_r = np.ediff1d(rpks)
    diff_r  = diff_r[diff_r < 600] # otherwise heart rate less than 40 bpm, 30 bpm -> 600
    #diff_r  = diff_r[diff_r > 60] # otw hr greater than 200 bpm, 81 would correpond to 220 bpm
    #min_dist = min(np.min(diff_r), min_dist)
    #max_dist = max(np.max(diff_r), max_dist)
    if (len(diff_r) == 0):
        diff_r = np.array([250])
    sigma = np.std(diff_r)
    med  = np.median(diff_r)
    avg = np.mean(diff_r)
    return np.array([sigma, med, avg])
    
rr_stats = np.apply_along_axis(getRRstats, axis = 1, arr = r_peaks) 
print(max_dist)
print(min_dist)

-5
1000


In [8]:
def getQRSwidth(mix_sig):
    rpks = mix_sig[0:212]
    sig = mix_sig[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    sig  = sig[sig < MAX_SENT]
    
    info = ecg.extract_heartbeats(signal=sig, rpeaks=rpks, sampling_rate= 300, before=0.30, after=0.30)
    beats = info[0]
    rpks = info[1]
    rrints = []
    for idx, bt in enumerate(beats):
        left =  88
        right = 92
        while ((left > 0) and bt[left] < bt[left + 1]):
            left -= 1
        while ((right < (len(bt)-1)) and bt[right] > bt[right +1]):
            right += 1
        rrints.append(right - left)
    res = np.array([np.mean(rrints), np.median(rrints)])
    return res
    ## go down to left and down to right until until hit bottom
    ## get the mean and median across all beats
mix_data = np.concatenate((r_peaks, X_), axis = 1)
qrs_stats = np.apply_along_axis(getQRSwidth, axis = 1, arr =mix_data)

In [10]:
def slopestat(mix_data2):
    ## inside the R-R interval look at the slope values
    ## skip the upward and donward slopes by cutting off 20 samples from the extremities
    ## get the slopes standard deviation of slope for each RR interval and average across all intervals
    ## get the number of times the sign of the slope changes for each R-R intevral divided by amount of time
    rpks = mix_data2[0:212]
    sig = mix_data2[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    sig  = sig[sig < MAX_SENT]
    if (len(rpks) <= 1): 
        return np.array([0]*6)
    stddevs = []
    slch = []
    abs_sl = np.array([])
    for i in range(len(rpks)-1):
        peak1 = rpks[i]
        peak2 = rpks[i+1]
        if (peak2 - peak1 < 40):
            continue
        loff = 18 + peak1
        roff = peak2 - 18
        seg = sig[loff:roff]
        slops = np.ediff1d(seg)
        stddevs.append(np.std(slops))
        abs_sl = np.append(abs_sl, np.absolute(slops))
        #nbr times slopes changes divided by seg len
        cnt = (np.diff(np.sign(slops)) != 0)*1
        cnt_ = float(len(cnt[cnt > 0]))
        slch.append((100*cnt_)/len(seg))
    
    res =  np.array([np.mean(slch), np.median(slch), np.mean(stddevs),np.median(stddevs), np.mean(abs_sl), np.median(abs_sl)])
    return res
    
slop_feats = np.apply_along_axis(slopestat, axis = 1, arr = mix_data2)

In [11]:
def compute_wavelet_descriptor(beat, family, level):
    wave_family = pywt.Wavelet(family)
    coeffs = pywt.wavedec(beat, family, level=level)
    return coeffs[0]

def DWT_beat(mix_sig):
    rpks = mix_sig[0:212]
    sig = mix_sig[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    sig  = sig[sig < MAX_SENT]
    
    info = ecg.extract_heartbeats(signal=sig, rpeaks=rpks, sampling_rate= 300, before=0.30, after=0.30)
    
    beat_pre = info[0]
    bts_cnt = min(15, len(beat_pre))
    wv = np.zeros((1, 23))[0]
    for i in range(bts_cnt):
        wv = np.add(wv, compute_wavelet_descriptor(beat_pre[i], 'db1', 3))
    wv = wv/bts_cnt
    return wv

dwt_feats = np.apply_along_axis(DWT_beat, axis = 1, arr = mix_data2)

In [12]:
def trans_PCA(X, variance):
    pca_ = PCA(variance)
    sc = StandardScaler()
    sc.fit(X)
    Xsc = sc.transform(X)
    pca_.fit(Xsc)
    X_ = pca_.transform(Xsc)
    return (X_, pca_, sc)
    ## do pca with the given variance
    ## return the transformed X as well as the PCA transformer (X, pca_variable)
    
Xtr, p_var, sc = trans_PCA(dwt_feats, 0.90) # try changing to 0.95 perhaps later

In [41]:
def devpreR(mix_sig):   ## added post R also
    rpks = mix_sig[0:212]
    sig = mix_sig[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    sig  = sig[sig < MAX_SENT]
    
    # previously was 0.30 - 0.0
    info = ecg.extract_heartbeats(signal=sig, rpeaks=rpks, sampling_rate= 300, before=0.30, after=0.4)
    tmpls = info[0]
    width = len(tmpls[0])
    tmpls = tmpls[:, 0:int(3*width/7)]
    nbrbt = len(tmpls)
    matr = np.array([[0]*60]*nbrbt)  # 60 inst. of 55 could also work
    for idx, bt in enumerate(tmpls):
        matr[idx] = bt[0:60]
    std60 = np.std(matr, axis= 0)
    
    tmpls2 = info[0]
    tmpls2 = tmpls2[:, int(3*width/7):]
    nbrbt = len(tmpls2)
    width = len(tmpls2[0])/2
    matr = np.array([[0]*60]*nbrbt)  # 60 inst. of 55 could also work
    for idx, bt in enumerate(tmpls2):
        matr[idx] = (bt[(int(width)-1):])[0:60]
    stdpost = np.std(matr, axis= 0)
    return np.append(std60, stdpost)

devvec = np.apply_along_axis(devpreR, axis = 1, arr = mix_data2)   

In [16]:
# Compute the HOS descriptor for a beat
# Skewness (3 cumulant) and kurtosis (4 cumulant)
def compute_hos_descriptor(mix_sig):
    rpks = mix_sig[0:212]
    sig = mix_sig[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    sig  = sig[sig < MAX_SENT]
    
    beats, _ = ecg.extract_heartbeats(signal=sig, rpeaks=rpks, sampling_rate= 300, before=0.25, after=0.25)
    #print(beats.shape)
    wnlen = len(beats[0])
    
    n_intervals = 5
    beat = np.mean(beats, axis = 0) 
    lag = int(round( (wnlen)/ n_intervals))
    hos_b = np.zeros(( (n_intervals-1) * 2))
    for i in range(0, n_intervals-1):
        pose = (lag * (i+1))
        interval = beat[(pose -int(round(lag/2)) ):(pose + int(round(lag/2)))]
        
        # Skewness  
        hos_b[i] = scipy.stats.skew(interval, axis = 0, bias=True)

        if np.isnan(hos_b[i]):
            hos_b[i] = 0.0
            
        # Kurtosis
        hos_b[(n_intervals-1) +i] = scipy.stats.kurtosis(interval, 0, False, True)
        if np.isnan(hos_b[(n_intervals-1) +i]):
            hos_b[(n_intervals-1) +i] = 0.0
    return hos_b

hos_feats = np.apply_along_axis(compute_hos_descriptor, axis = 1, arr = mix_data2)   

In [21]:
def kmeansrr(rpks):
    rpks = rpks[rpks < MAX_SENT]
    diff_r = np.ediff1d(rpks)
    kmeans = KMeans(n_clusters=2, tol = 40, random_state=0).fit(diff_r.reshape(-1, 1))
    labels = kmeans.labels_
    #print(labels.shape)
    length = len(labels)
    len0 = (len(labels[labels < 1]))/length
    len1 = (length - len0)/length
    cntrs = kmeans.cluster_centers_
    return abs(len0*cntrs[0]-len1*cntrs[1])

kmeans = np.apply_along_axis(kmeansrr, axis = 1, arr = r_peaks) 

In [29]:
def autoregression(mix_sig):  # This one should work
    rpks = mix_sig[0:212]
    sig = mix_sig[212:]
    
    rpks  = np.array(rpks[rpks < MAX_SENT], dtype = int)
    real_sig  = sig[sig < MAX_SENT]
    
    qrs, _ = ecg.extract_heartbeats(signal=real_sig, rpeaks=rpks, sampling_rate= 300, before=0.056, after=0.056)
    stt, _ = ecg.extract_heartbeats(signal=real_sig, rpeaks=rpks, sampling_rate= 300, before=0, after=0.39)
    
    qrs_avg = np.mean(qrs, axis = 0)
    stt_avg = np.mean(stt, axis = 0)
    
    start_pos = (20/140)*len(stt_avg)
    stt_avg = stt_avg[int(start_pos):]
    _, qrs_arcoeffs, _, _, _ = levinson_durbin(qrs_avg, nlags=9, isacov=False)
    _, stt_arcoeffs, _, _, _ = levinson_durbin(stt_avg, nlags=10, isacov=False)
    return np.append(qrs_arcoeffs, stt_arcoeffs)
    
ar_coeffs = np.apply_along_axis(autoregression, axis = 1, arr = mix_data2) 

In [43]:
def noise_highfreq(rsig): #baseline wander is not considered noise
    sig_ = rsig[rsig < MAX_SENT]
    coeffs = wavedec(sig_, 'db6', level=9)
    cApprox, *cdetail = coeffs
    #new_coeffs = [np.zeros_like(cApprox), cdetail[0], cdetail[1], cdetail[2], cdetail[3], cdetail[4], cdetail[5], cdetail[6], np.zeros_like(cdetail[7]), np.zeros_like(cdetail[8])]
    # first and second level detail contain all the noise information detectable for sampling frequency 300 Hz (Nyquist Sampling Theorem)
    noise_coeffs = [cdetail[7], cdetail[8]]
    return noise_coeffs
noise_coeffs = np.apply_along_axis(noise_highfreq, axis = 1, arr = X_)

In [44]:
print(rr_stats.shape)
print(qrs_stats.shape)
print(slop_feats.shape)
print(Xtr.shape)
print(devvec.shape)
print(kmeans.shape)
print(hos_feats.shape)
print(ar_coeffs.shape)
print(noise_coeffs.shape)

(4947, 3)
(4947, 2)
(4947, 6)
(4947, 11)
(4947, 120)
(4947, 1)
(4947, 8)
(4947, 6)
(4947, 2)


In [31]:
# total of 82 feats

In [ ]:
Xtrain = np.concatenate((rr_stats, qrs_stats, slop_feats, Xtr, devvec, hos_feats, kmeans, ar_coeffs, noise_coeffs), axis = 1)
print(Xtrain.shape)

In [ ]:
# do same transformation on test X
rT_peaks = np.apply_along_axis(getRpeaks, axis = 1, arr = Xt)
rrT_stats = np.apply_along_axis(getRRstats, axis = 1, arr = rT_peaks) 
mixT_data = np.concatenate((rT_peaks, Xt), axis = 1)
qrsT_stats = np.apply_along_axis(getQRSwidth, axis = 1, arr =mixT_data)
XT_filt = np.apply_along_axis(filtersig, axis = 1, arr = Xt)
mixT_data2 = np.concatenate((rT_peaks, XT_filt), axis = 1)
slopT_feats = np.apply_along_axis(slopestat, axis = 1, arr = mixT_data2)
dwtT_feats = np.apply_along_axis(DWT_beat, axis = 1, arr = mixT_data2)
devTvec = np.apply_along_axis(devpreR, axis = 1, arr = mixT_data2)
XtrT = sc.transform(dwtT_feats)
XtrT = p_var.transform(XtrT)
hosT_feats = np.apply_along_axis(compute_hos_descriptor, axis = 1, arr = mixT_data2)   
kmeansT = np.apply_along_axis(kmeansrr, axis = 1, arr = rT_peaks) 
arr_coeffsT = np.apply_along_axis(autoregression, axis = 1, arr = mixT_data2) 
noiseT_coeffs = np.apply_along_axis(noise_highfreq, axis = 1, arr = Xt) 


Xtest = np.concatenate((rrT_stats, qrsT_stats, slopT_feats, XtrT, devTvec, hosT_feats, kmeansT, arr_coeffsT, noiseT_coeffs), axis = 1)

In [45]:
noiseT_coeffs = np.apply_along_axis(noise_highfreq, axis = 1, arr = Xt) 

In [46]:
print(noiseT_coeffs.shape)

(3411, 2)


## This is where you do your XGboost magic
# You only need to touch Xtest and Xtrain
## Once your code predicts and produces a csv, send me the predicted probabilities, the csv file as well as the optimal parameters (try also using lightgbm using the 'balanced' parameter and catboost using ['Balanced', 'SqrtBalanced'] parameter )
# CAUTION: 
## This code does not predict class 3. Class 3 is predicted by a noise classifier (one versus all). Both are combined using soft voting, i.e. if noise classifier predicts sample 4 is 1 (=class 3) with probability 0.63 but this classifier predicts it is class 2 with probability 0.87 then the final classification will be class 2.  

In [ ]:
preds012 = [] # This is where you will store your predictions for class 0, 1, and 2 **only**

#### Your code starts here 

In [ ]:
def combinePredictions(preds012, preds3):
    final_preds = [0]*len(Xt)
    ctr = 0
    for idx, el in enumerate(preds3):
        if (preds[3] > 0.5):  
            final_preds[idx] = 3
        else:
            final_preds[idx] = preds012[ctr]
            ctr += 1
    return final_preds

In [ ]:
fin_preds = combinePredictions(preds012, preds3)
dfPredictions = pd.DataFrame()
dfPredictions.index.name = "id"
dfPredictions.to_csv("denoisedpreds.csv", header = ['y'], index=True)